In [24]:
import pyspark
print(pyspark.__version__)

3.5.1


In [25]:
from pyspark.sql import SparkSession
from pyspark.sql. functions import col, when, avg, sum as spark_sum, count, expr

In [26]:
# Create Spark session

spark = SparkSession.builder.appName("FakeOrdersEDA").getOrCreate()

In [27]:
# Load the data from a CSV

df = spark.read.csv("project_dataset/python_raw_data/fake_orders_test_updated.csv/updated_dataset.csv", header=True, inferSchema=True)

In [46]:
# Show the first rows of the dataframe
df.show(10)

+--------+---------------------+------------+-------------+---------------+--------------+--------+------------------+--------------------+--------------------+
|order_id|activation_time_local|country_code|store_address|   final_status|payment_status|products|    products_total|purchase_total_price|   authorized_or_not|
+--------+---------------------+------------+-------------+---------------+--------------+--------+------------------+--------------------+--------------------+
|31503775|  2019-03-01 11:43:08|          ES|        15871|DeliveredStatus|          PAID|       1|              1.85|               14.02|    under_authorized|
|31503965|  2019-03-01 11:43:08|          ES|        15871|DeliveredStatus|          PAID|       3|              6.15|               12.21|    under_authorized|
|31636675|  2019-03-01 08:58:01|          AR|        61807|DeliveredStatus|          PAID|       4|1.1800000000000002|                9.76|    under_authorized|
|31724509|  2019-03-01 16:43:04|  

In [29]:
df.printSchema()

root
 |-- order_id: integer (nullable = true)
 |-- activation_time_local: timestamp (nullable = true)
 |-- country_code: string (nullable = true)
 |-- store_address: integer (nullable = true)
 |-- final_status: string (nullable = true)
 |-- payment_status: string (nullable = true)
 |-- products: integer (nullable = true)
 |-- products_total: double (nullable = true)
 |-- purchase_total_price: double (nullable = true)
 |-- authorized_or_not: string (nullable = true)



You’ve been given a data set about orders coming from Fake partners in the app. Fake partners are the stores that are not integrated with IronHack Delivery directly so our content team manages their product catalog and prices for them. Fake partner orders are charged to the customer upon delivery and in many cases there is a mismatch between the total amount at checkout in the app (products_total) and what the courier pays at the store (purchase_total_price) causing many problems. When the products_total is lower than purchase_total_price we call them under-authorized orders, otherwise is a correctly authorized order. We want to move away from charge-on-delivery to an authorize-and-capture model but we first need to understand the price fluctuation of past orders to know the risk of doing so.

Your task is to perform an EDA process (R/Python) with this data to answer the following questions:

What percent of orders are under-authorized?

What percent of orders would be correctly authorized with incremental authorization (+20%) on the amount at checkout?

Are there differences when split by country?

For the remainder of orders that would be outside of incremental authorization, what values would be necessary to capture the remaining amount?

Which stores are the most problematic in terms of orders and monetary value?

For under-authorized orders, is there a correlation between the difference in the prices and the cancellation of the order? In other words: Is an order more likely to be cancelled as the price difference increases?

In [30]:
# Register the DataFrame as a SQL temporary view
df.createOrReplaceTempView("orders")

In [31]:
# What percent of orders are under-authorized?

# SQL query to identify under-authorized orders
query = """
SELECT 
    COUNT(*) AS total_orders,
    SUM(CASE WHEN products_total < purchase_total_price THEN 1 ELSE 0 END) AS under_authorized_orders
FROM orders
"""

# Execute the query
result = spark.sql(query)

# Calculate the percentage of under-authorized orders
result = result.withColumn("percentage_under_authorized", 
                           (result["under_authorized_orders"] / result["total_orders"]) * 100)

# Show the result
result.show()


+------------+-----------------------+---------------------------+
|total_orders|under_authorized_orders|percentage_under_authorized|
+------------+-----------------------+---------------------------+
|       60400|                  34781|          57.58443708609271|
+------------+-----------------------+---------------------------+



In [32]:
# What percent of orders would be correctly authorized with incremental authorization (+20%) on the amount at checkout?

query = """
SELECT
  (SUM
   (CASE
      WHEN PRODUCTS_TOTAL * 1.2 >= purchase_total_price THEN 1
      ELSE 0
    END) * 100 / COUNT (*)
    ) AS percent_authorized_when_incremented
   FROM orders;
"""

# Execute the query
result = spark.sql(query)

# Show the result
result.show()

+-----------------------------------+
|percent_authorized_when_incremented|
+-----------------------------------+
|                  69.03476821192054|
+-----------------------------------+



In [34]:
# Are there differences when split by country?

query = """
SELECT
    country_code,
    (SUM(CASE
            WHEN (products_total * 1.20) >= purchase_total_price THEN 1
            ELSE 0
         END) * 100.0 / COUNT(*)) AS percent_correctly_authorized_with_increment
FROM
    orders
GROUP BY
    country_code;
"""

# Execute the query
result = spark.sql(query)

# Show the result
result.show()

+------------+-------------------------------------------+
|country_code|percent_correctly_authorized_with_increment|
+------------+-------------------------------------------+
|          CI|                          66.66666666666667|
|          UA|                          72.66326778344258|
|          RO|                          72.17351176742040|
|          PT|                          63.91639163916392|
|          CL|                          12.95289855072464|
|          BR|                          50.00000000000000|
|          GT|                          79.71781305114638|
|          ES|                          71.49773543651413|
|          EC|                          62.59968102073365|
|          TR|                          68.88783868935098|
|          CR|                          79.85546522131888|
|          FR|                          67.32209737827715|
|          PA|                          66.92991115498519|
|          IT|                          70.0854700854700

In [37]:
# For the remainder of orders that would be outside of incremental authorization, what values would be necessary to capture the remaining amount?

# Part 1

query = """
SELECT
(SUM(
   CASE
      WHEN (PRODUCTS_TOTAL * 1.2) < purchase_total_price THEN 1
      ELSE 0
    END) * 100 / COUNT (*)) AS percent_NOT_authorized_when_incremented
   FROM orders;
"""

# Execute the query
result = spark.sql(query)

# Show the result
result.show()

+---------------------------------------+
|percent_NOT_authorized_when_incremented|
+---------------------------------------+
|                      30.96523178807947|
+---------------------------------------+



In [38]:
# Part 2

query = """
SELECT
  order_id, products_total, purchase_total_price, (purchase_total_price - products_total * 1.2) AS additional_required
  FROM orders
  WHERE (products_total * 1.2) < purchase_total_price;
"""

result = spark.sql(query)
result.show()

+--------+------------------+--------------------+-------------------+
|order_id|    products_total|purchase_total_price|additional_required|
+--------+------------------+--------------------+-------------------+
|31503775|              1.85|               14.02| 11.799999999999999|
|31503965|              6.15|               12.21|  4.830000000000001|
|31636675|1.1800000000000002|                9.76|              8.344|
|31839133|              1.29|               11.19|              9.642|
|31868928|             24.65|               31.68| 2.1000000000000014|
|31890972|               6.8|                21.8|              13.64|
|31908582|              8.07|               14.88| 5.1960000000000015|
|31934292|              3.86|               24.27| 19.637999999999998|
|31949773|              6.21|               11.46|  4.008000000000001|
|31950805|             15.27|               18.78|0.45600000000000307|
|31965637|              3.44|   6.890000000000001| 2.7620000000000005|
|31968

In [41]:
# Which stores are the most problematic in terms of orders and monetary value?

# By number of orders

query = """
SELECT store_address, COUNT (*) as number_of_problematic_orders
FROM orders
WHERE products_total < purchase_total_price
GROUP BY store_address
ORDER BY number_of_problematic_orders DESC;
"""

result = spark.sql(query)
result.show(10)

+-------------+----------------------------+
|store_address|number_of_problematic_orders|
+-------------+----------------------------+
|        28671|                         448|
|        28712|                         217|
|        28286|                         204|
|        14455|                         179|
|        11694|                         160|
|        28669|                         158|
|        12513|                         126|
|        55206|                         120|
|        27635|                         116|
|        62935|                         111|
+-------------+----------------------------+
only showing top 10 rows



In [43]:
# By monetary value

query = """
SELECT store_address, SUM (purchase_total_price - products_total) as total_discrepancy
FROM orders
WHERE products_total < purchase_total_price
GROUP BY store_address
ORDER BY total_discrepancy DESC;
"""

result = spark.sql(query)
result.show(10)

+-------------+------------------+
|store_address| total_discrepancy|
+-------------+------------------+
|        10264|           1781.13|
|        28669|1185.6999999999998|
|        65896|           1011.47|
|         9855|            907.03|
|        14455| 791.4099999999999|
|        28671|             775.8|
|         9854|            752.74|
|        17327|            737.33|
|        10358|            666.04|
|        65898| 651.6299999999999|
+-------------+------------------+
only showing top 10 rows



In [51]:
# For under-authorized orders, is there a correlation between the difference in the prices and the cancellation of the order?
# In other words: Is an order more likely to be cancelled as the price difference increases?

# Part 1

query = """
SELECT order_id, purchase_total_price - products_total AS price_difference,
CASE
  WHEN final_status = 'CanceledStatus' THEN 1
  ELSE 0
END AS is_cancelled
FROM orders
WHERE products_total < purchase_total_price;
"""

result = spark.sql(query)
result.show()

+--------+-------------------+------------+
|order_id|   price_difference|is_cancelled|
+--------+-------------------+------------+
|31503775|              12.17|           0|
|31503965| 6.0600000000000005|           0|
|31636675|               8.58|           0|
|31724509| 1.4499999999999993|           0|
|31839133|  9.899999999999999|           0|
|31868928|  7.030000000000001|           0|
|31890972|               15.0|           0|
|31908582| 6.8100000000000005|           0|
|31934292|              20.41|           0|
|31936061|0.21999999999999886|           0|
|31949773|  5.250000000000001|           0|
|31950653| 2.6199999999999974|           0|
|31950805| 3.5100000000000016|           0|
|31960607|0.16000000000000014|           0|
|31965637| 3.4500000000000006|           0|
|31966752|0.34999999999999964|           0|
|31968802|               2.79|           0|
|31969123| 3.3999999999999995|           0|
|31969645|               3.28|           0|
|31970513|0.48999999999999977|  

In [ ]:
# Stop the Spark session
spark.stop()